In [1]:
import torch
import yaml
import argparse
import time
from pathlib import Path
from rich.console import Console
from rich.table import Table
from torchvision.datasets import *
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import GradScaler, autocast
import torchvision.transforms as transforms

from datasets.transforms import get_train_transforms, get_val_transforms
from models import *
from utils.losses import LabelSmoothCrossEntropy, CrossEntropyLoss
from utils.utils import fix_seeds, setup_cudnn, create_progress_bar
from utils.metrics import compute_accuracy

from torchvision.datasets import ImageFolder
from typing import Optional, Callable
from pathlib import Path

import torch
import os 
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import cv2
import rasterio 

#-- Console 
console = Console()


#-- load for map your city 
#--- path
input_path = "/mnt/hdd/eric/.tmp_ipy/00.Data/Map_Your_City/building-age-dataset/"
train_path = input_path + "train/data/"
test_path = input_path + "test/data/"

#--- device
device = torch.device("cuda:0")
print('Device: ' + str(device))   

#--- Load csv files
test_df = pd.read_csv(input_path + "test/test-set.csv") 

#--- for submission 
names_data_test = os.listdir(test_path)

FILTERED_data = {} 
IDS = names_data_test
for idx,ID in enumerate(IDS):
    test_img_path = os.path.join(test_path + ID + '/street.jpg')
    if os.path.exists( test_img_path ):
        FILTERED_data[idx]=ID


#--- dataset class 
class Map_Dataset(torch.utils.data.Dataset):      
    def __init__(self, list_IDs,transform): 
        self.list_IDs = list_IDs
        self.transform = transform
        
    def __len__(self):
        return len(self.list_IDs)
    
    def __getitem__(self, index): 
        ID = self.list_IDs[index] 
        X1 = cv2.imread(test_path + ID + '/street.jpg')
        X1 = cv2.resize(X1, (256, 256))
        X1 = np.asarray(X1)
        #X1 = np.transpose(X1,[1, 2, 0])
        X1 = self.transform(X1)
        #X1 = cv2.resize(X1, (256, 256)) 
        
        #X2 = cv2.imread(train_path + ID + '/orthophoto.tif')  
        #X2 = cv2.resize(X2, (256, 256)) 
        
        #X3 = rasterio.open(train_path + ID + '/s2_l2a.tif').read() 
        #X3 = np.transpose(X3, [1, 2, 0]) 
        
        #y = int(open(test_path + ID + '/label.txt', "r").read())
        return X1 
    
#---

parser = argparse.ArgumentParser()
parser.add_argument('--cfg', type=str, default='configs/finetune.yaml')
args = parser.parse_args()
cfg = argparse.Namespace(**yaml.load(open(args.cfg), Loader=yaml.SafeLoader))

#-- device
device = "cuda:1"

# initialize model and load imagenet pretrained
model = eval(cfg.MODEL)(cfg.VARIANT, cfg.PRETRAINED, cfg.CLASSES_NUM, cfg.IMAGE_SIZE)

#--- load model 
saved_model_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/07.Challenge/03.ImageNet_SOTAs/sota-backbones/output/VAN_L_epoch_2.pth"
model.load_state_dict(torch.load(saved_model_path))
model = model.to(device)

# submit_loader
data_transform = transforms.Compose([  
        transforms.ToTensor() 
    ])

#--- train and valid selection need
testset = Map_Dataset(FILTERED_data,data_transform)  
num_workers = 8 

# dataloader 
testloader = DataLoader(testset, 1, shuffle=False, num_workers=num_workers, pin_memory=True)


# Testing
def test(model):                      
    print('Now testing')                                  
    model.eval()                        
    running_loss = 0.0  
    running_correct = 0

    model_predictions = []
    #with torch.no_grad():
    cnt =0
    for idx,batch in enumerate(testloader):  
        print(f"{cnt} / {len(testloader)}")
        img = batch
        img = img.to(device)
        outputs = model(img) 
        _, preds = torch.max(outputs, 1)
        model_predictions.append(preds)
        cnt += 1 
    return model_predictions

/mnt/hdd/eric/.conda/envs/mah_v1/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda:0


usage: ipykernel_launcher.py [-h] [--cfg CFG]
ipykernel_launcher.py: error: unrecognized arguments: --f=/mnt/hdd/eric/.local/share/jupyter/runtime/kernel-v2-1734414Dv8YTKDemYGn.json


SystemExit: 2

/mnt/hdd/eric/.conda/envs/mah_v1/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#---
predictions_ = test(model)
